<a href="https://colab.research.google.com/github/wuphwu/IDS/blob/main/SCARF_for_CACTUs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SCARF + CACTUs Meta-Learning Pipeline

這個 notebook 示範如何使用 SCARF 對 tabular data 預訓練 encoder，
接著透過 CACTUs 利用 k-means clustering 建構 meta-learning 任務。

# 資料處理

In [33]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
hassan06_nslkdd_path = kagglehub.dataset_download('hassan06/nslkdd')

print('Data source import complete.')
print(f"資料集下載路徑：{hassan06_nslkdd_path}")


Data source import complete.
資料集下載路徑：/kaggle/input/nslkdd


In [34]:
import os

# 使用下載的路徑來建構檔案的完整路徑
file_path = os.path.join(hassan06_nslkdd_path, 'KDDTrain+.txt')

In [35]:
import os
import pandas as pd
# 使用下載的路徑來建構檔案的完整路徑
file_path = os.path.join(hassan06_nslkdd_path, 'KDDTrain+.txt')

df_0 = df_0 = pd.read_csv(file_path, sep=',', header=None, encoding='latin-1')
df= df_0.copy()
df.head() # 顯示資料集前五行

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
1,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
2,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
3,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
4,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21


In [36]:
# 定義資料集的欄位名稱
columns = (['duration'
,'protocol_type'
,'service'
,'flag'
,'src_bytes'
,'dst_bytes'
,'land'
,'wrong_fragment'
,'urgent'
,'hot'
,'num_failed_logins'
,'logged_in'
,'num_compromised'
,'root_shell'
,'su_attempted'
,'num_root'
,'num_file_creations'
,'num_shells'
,'num_access_files'
,'num_outbound_cmds'
,'is_host_login'
,'is_guest_login'
,'count'
,'srv_count'
,'serror_rate'
,'srv_serror_rate'
,'rerror_rate'
,'srv_rerror_rate'
,'same_srv_rate'
,'diff_srv_rate'
,'srv_diff_host_rate'
,'dst_host_count'
,'dst_host_srv_count'
,'dst_host_same_srv_rate'
,'dst_host_diff_srv_rate'
,'dst_host_same_src_port_rate'
,'dst_host_srv_diff_host_rate'
,'dst_host_serror_rate'
,'dst_host_srv_serror_rate'
,'dst_host_rerror_rate'
,'dst_host_srv_rerror_rate'
,'attack'
,'level'])

df.columns = columns

# y lable one-hot encoding
因應SCARF執行上的需要，進行attack欄位編碼,避免字串

In [37]:
# 用LabelEncoder()把y的資料轉換成整數型態
from sklearn import preprocessing
le=preprocessing.LabelEncoder()
clm=['attack']
for x in clm:
    df[x]=le.fit_transform(df[x])

# 分割資料集

In [38]:
from sklearn.model_selection import train_test_split

seed = 1234

data = df.copy()
data, target = data.drop(['level','attack'], axis=1), data["attack"]
train_data, test_data, train_target, test_target = train_test_split(
    data, target, test_size=0.2, stratify=target, random_state=seed
)

# 類別型欄位Frequency Encoding(要分割完測試資料再做)

就是我們把Categorical Feature裡面每個類別出現的數量，當成他的數值。

In [39]:
!pip install category_encoders
from category_encoders import CountEncoder

In [40]:
# use target encoding to encode categorical features
categorical_features_to_encode = ['protocol_type', 'service', 'flag']
enc = CountEncoder(cols=categorical_features_to_encode)

# transform the datasets
X_train_e = enc.fit_transform(train_data)
X_test_e = enc.transform(test_data)

# 特徵工程

In [41]:
from sklearn.feature_selection import mutual_info_classif

mutual_info = mutual_info_classif(X_train_e, train_target)
mutual_info = pd.Series(mutual_info)
mutual_info.index = X_train_e.columns
mutual_info.sort_values(ascending=False)

,0
src_bytes,0.932717
service,0.762072
flag,0.676077
diff_srv_rate,0.657273
same_srv_rate,0.622332
dst_host_diff_srv_rate,0.612758
dst_host_same_srv_rate,0.563243
count,0.559766
dst_host_srv_count,0.540867
dst_bytes,0.502502


In [42]:
# **新增：根據 mutual_info 選擇前 30 個特徵**
top_features = mutual_info.head(30).index

# 使用選定的特徵來更新 X_train 和 X_test
X_train = X_train_e[top_features]
X_test = X_test_e[top_features]

# Scaling

In [43]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)

# SCARF

In [44]:
!pip install git+https://github.com/wuphwu/pytorch-scarf.git
!pip install torch

  Cloning https://github.com/wuphwu/pytorch-scarf.git to /tmp/pip-req-build-4oixm430
  Running command git clone --filter=blob:none --quiet https://github.com/wuphwu/pytorch-scarf.git /tmp/pip-req-build-4oixm430
  Resolved https://github.com/wuphwu/pytorch-scarf.git to commit 538be476bd57562f039e0638081e1998731bd86e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [45]:
# 1. 載入資料與必要套件
import torch
from torch.optim import Adam
from scarf.loss import NTXent
from scarf.model import SCARF
from torch.utils.data import DataLoader
import numpy as np # Import the numpy library
from scarf.dataset import SCARFDataset


In [46]:
# 2. to torch dataset
train_ds = SCARFDataset(X_train_s, train_target.to_numpy())
test_ds = SCARFDataset(X_test_s, test_target.to_numpy())

print(f"Train set: {train_ds.shape}")
print(f"Test set: {test_ds.shape}")

Train set: (100778, 30)
Test set: (25195, 30)


| 參數名稱                            | 所屬網路             | 說明                                  |
| ------------------------------- | ---------------- | ----------------------------------- |
| `input_dim`                     | `f`              | 輸入資料的特徵數，即原始 tabular 資料的維度          |
| `dim_hidden_encoder`            | `f`              | encoder（`f`）每層隱藏層的維度                |
| `num_hidden_encoder`            | `f`              | encoder（`f`）的隱藏層數量                  |
| `dim_hidden_head`               | `g`              | projection head（`g`）每層隱藏層的維度        |
| `num_hidden_head`               | `g`              | projection head（`g`）的層數             |
| `features_low`, `features_high` | 🚫與網路結構無關        | 指示哪些欄位是連續變數、離散變數（通常用來處理破壞策略）        |
| `corruption_rate`               | 🚫與網路結構無關        | SCARF 特有的參數，表示特徵破壞的比例（用於產生自監督 view） |
| `dropout`                       | `f` + `g`（視實作而定） | 用於正則化，可能同時應用在 encoder 和 head 中      |


In [47]:
# 首先確保安裝了 pytorch-lightning
# 如果還沒安裝，請執行下面的指令
!pip install pytorch_lightning

# dim_hidden_encoder 選擇

| 嵌入維度     | 建議    | 適用情境      | 備註                     |
| -------- | ----- | --------- | ---------------------- |
| **15**   | ✅ 可行  | 保持原維度     | 若你想與原資料對比              |
| **32**   | ✅ 推薦  | 通用設定      | 充分保留結構又避免維度詛咒          |
| **64**   | ✅ 推薦  | 當資料量夠大    | 提升表達能力                 |
| **128+** | ⚠️ 小心 | 僅適合樣本數很多時 | 否則會過擬合 & clustering 崩潰 |
| **< 15** | ✅ 可測試 | 若你想降維壓縮資訊 | 注意避免損失語義結構             |


In [48]:
import torch
from torch.optim import Adam
from scarf.loss import NTXent
from scarf.model import SCARF
from torch.utils.data import DataLoader
import numpy as np
from scarf.dataset import SCARFDataset
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from torch.utils.data import Dataset # 確認有匯入 Dataset

# ✅ 3. 將 SCARF 模型包裝成 PyTorch Lightning Module
class SCARFLightning(pl.LightningModule):
    def __init__(self, input_dim, dim_hidden_encoder, num_hidden_encoder,
                 dim_hidden_head, num_hidden_head, features_low, features_high,
                 corruption_rate, dropout, learning_rate=0.001):
        super().__init__()
        # 保存超參數
        self.save_hyperparameters()

        # 初始化 SCARF 模型
        self.scarf_model = SCARF(
            input_dim=input_dim,
            dim_hidden_encoder=dim_hidden_encoder,
            num_hidden_encoder=num_hidden_encoder,
            dim_hidden_head=dim_hidden_head,
            num_hidden_head=num_hidden_head,
            features_low=features_low,
            features_high=features_high,
            corruption_rate=corruption_rate,
            dropout=dropout
        )

        # 初始化損失函數
        self.criterion = NTXent()

    def forward(self, x):
        # LightningModule 通常只需要實現一步 forward
        # 訓練步會在 training_step 中處理
        emb_anchor, emb_positive = self.scarf_model(x)
        return emb_anchor, emb_positive

    def training_step(self, batch, batch_idx):
        # 訓練的單一步驟
        x = batch # SCARFDataset 只回傳 data，沒有 target 在訓練中直接使用
        emb_anchor, emb_positive = self.forward(x)
        loss = self.criterion(emb_anchor, emb_positive)
        self.log('train_loss', loss) # 記錄訓練損失
        return loss

    def configure_optimizers(self):
        # 配置優化器
        optimizer = Adam(self.parameters(), lr=self.hparams.learning_rate, weight_decay=1e-4)
        return optimizer

# ✅ 4. (可選) 將資料集和 DataLoader 包裝成 PyTorch Lightning DataModule
# 這不是必須的，但對於更複雜的資料 pipeline 很有用
class SCARFDataModule(pl.LightningDataModule):
    def __init__(self, train_dataset: Dataset, test_dataset: Dataset, batch_size: int = 128):
        super().__init__()
        self.train_dataset = train_dataset
        self.test_dataset = test_dataset
        self.batch_size = batch_size

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        # SCARF 預訓練通常沒有驗證集，可以回傳 None 或測試集 DataLoader
        # 我們在這裡使用測試集作為驗證，但要注意這不是真正的驗證過程
        return DataLoader(self.test_dataset, batch_size=self.batch_size)

# 檢查訓練資料是否有常數特徵 (標準差為 0)
stds = np.std(X_train_s, axis=0)
constant_features_indices = np.where(stds == 0)[0]

if len(constant_features_indices) > 0:
    print(f"Warning: Found {len(constant_features_indices)} constant features in training data.")
    print(f"Removing features at indices: {constant_features_indices}")

    # 移除常數特徵
    X_train_s_filtered = np.delete(X_train_s, constant_features_indices, axis=1)
    X_test_s_filtered = np.delete(X_test_s, constant_features_indices, axis=1)
else:
    X_train_s_filtered = X_train_s
    X_test_s_filtered = X_test_s

# 使用過濾後的資料創建 SCARFDataset
train_ds = SCARFDataset(X_train_s_filtered, train_target.to_numpy())
test_ds = SCARFDataset(X_test_s_filtered, test_target.to_numpy())

print(f"Filtered Train set: {train_ds.shape}")
print(f"Filtered Test set: {test_ds.shape}")


# ✅ 5. 實例化模型和 DataModule
batch_size = 128
epochs = 1000

# 實例化 Lightning Module
pl_scarf = SCARFLightning(
    input_dim=train_ds.shape[1],
    dim_hidden_encoder=64,
    num_hidden_encoder=3,
    dim_hidden_head=64,
    num_hidden_head=3,
    features_low=train_ds.features_low, # 假設 features_low 和 features_high 在 SCARFDataset 中正確計算
    features_high=train_ds.features_high,
    corruption_rate=0.6,
    dropout=0.1,
    learning_rate=0.001 # 設置學習率
)

# 實例化 DataModule
datamodule = SCARFDataModule(train_ds, test_ds, batch_size=batch_size)

# ✅ 6. 使用 PyTorch Lightning Trainer 進行訓練
# 設置 Trainer，指定使用 'gpu' 加速
# 如果您有多個 GPU，可以指定 devices=n (n為GPU數量) 或 devices=[0, 1] 等
# 如果您確定有 GPU，可以直接寫 accelerator='gpu'
# 如果您想讓 Lightning 自動偵測和使用所有可用 GPU，可以寫 accelerator='gpu', devices='auto'
trainer = pl.Trainer(
    accelerator='gpu',        # 指定使用 GPU
    #devices=1 if torch.cuda.is_available() else 'auto', # 指定使用 1 個 GPU 如果可用，否則自動偵測 (如果沒有 GPU，會回到 CPU)
    max_epochs=epochs,
    # num_sanity_val_steps=0,  # 在預訓練中通常不需要驗證步
    # logger=None,             # 預設會使用 TensorBoard Logger，如果不需要可以設為 None
    # enable_checkpointing=False # 預設會儲存檢查點，如果不需要可以設為 False
    log_every_n_steps=50 # 可以設定每隔多少個訓練步驟記錄一次損失，預設是 50 [1, 2]
)

# 開始訓練
trainer.fit(pl_scarf, datamodule)

# ✅ 7. 儲存訓練好的 encoder 權重 (從 Lightning Module 中取出模型)
# 可以直接訪問 Lightning Module 中的 scarf_model 的 encoder
torch.save(pl_scarf.scarf_model.encoder.state_dict(), "encoder_f_lightning.pth")

print("SCARF pre-training finished using PyTorch Lightning.")

# 如果你想繪製損失，可以從 trainer.callback_metrics 中獲取，或者使用 logger
# (這裡省略繪圖部分，因為需要配置 logger)

INFO:pytorch_lightning.utilities.rank_zero:Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Removing features at indices: [19]
Filtered Train set: (100778, 29)
Filtered Test set: (25195, 29)


/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/configuration_validator.py:68: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name        | Type   | Params | Mode 
-----------------------------------------------
0 | scarf_model | SCARF  | 23.2 K | train
1 | criterion   | NTXent | 0      | train
-----------------------------------------------
23.2 K    Trainable params
0         Non-trainable params
23.2 K    Total params
0.093     Total estimated model params size (MB)
22        Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1000` reached.


SCARF pre-training finished using PyTorch Lightning.


# 待補繪製損失函數下降的圖

In [49]:
# 訓練結束後，損失和其他 metrics 會記錄在 logs/scarf_training/version_*/metrics.csv 文件中
# 您可以使用 pandas 讀取這個文件來繪製損失曲線

In [50]:
'''
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(loss_history)
ax.set_xlabel("epoch")
ax.set_ylabel("loss")
'''

'\nimport matplotlib.pyplot as plt\nfig, ax = plt.subplots(figsize=(10, 5))\nax.plot(loss_history)\nax.set_xlabel("epoch")\nax.set_ylabel("loss")\n'

# 取得train_ds的embeddings

In [51]:
'''
# 重新載入 f()（在其他地方用）
# 你必須重新建立 encoder 架構（要一樣的超參數）
encoder = Encoder(
    input_dim=train_ds.shape[1],
    dim_hidden=64,
    num_hidden=3,
    dropout=0.1
).to(device)

encoder.load_state_dict(torch.load("encoder_f.pth"))
encoder.eval()  # 推論用
'''

'\n# 重新載入 f()（在其他地方用）\n# 你必須重新建立 encoder 架構（要一樣的超參數）\nencoder = Encoder(\n    input_dim=train_ds.shape[1],\n    dim_hidden=64,\n    num_hidden=3,\n    dropout=0.1\n).to(device)\n\nencoder.load_state_dict(torch.load("encoder_f.pth"))\nencoder.eval()  # 推論用\n'

In [52]:
print(pl_scarf.scarf_model.encoder)
# ✅ 4. 取得train_ds 經過 f(x)的 embedding
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pl_scarf.to(device)
with torch.no_grad():
    x_tensor = torch.tensor(train_ds.data, dtype=torch.float32).to(device)  # PyTorch Dataset 物件轉換為 PyTorch tensor
    embeddings_train = pl_scarf.scarf_model.encoder(x_tensor).cpu().numpy() # 將從 GPU 計算得到的 embeddings tensor 複製回 CPU。這是因為後續的 numpy() 方法只能在 CPU tensor 上執行。
print("Embeddings shape:", embeddings_train.shape)

MLP(
  (0): Linear(in_features=29, out_features=64, bias=True)
  (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): Dropout(p=0.1, inplace=False)
  (4): Linear(in_features=64, out_features=64, bias=True)
  (5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): ReLU(inplace=True)
  (7): Dropout(p=0.1, inplace=False)
  (8): Linear(in_features=64, out_features=64, bias=True)
)
Embeddings shape: (100778, 64)


# 保存train .npz檔
{
  'X': ...  # 原始資料 (如 shape=[N, 64, 64, 3])
  'Y': ...  # 對應的lebal
  'Z': ...  # embeddings(如 shape=[N, 128])
}


In [53]:

# Save embeddings
output_dir = "data/scarf_encodings"
os.makedirs(output_dir, exist_ok=True)

# Get X=original Y=labels Z=embeddings
X_train= train_ds.data
Y_train= train_ds.target
Z_train= embeddings_train


# Save train split
train_path = os.path.join(output_dir, f"nslkdd_{Z_train.shape[1]}_train.npz")
print(f"Saving train embeddings to {train_path}")
np.savez(train_path,
          X=X_train,      # Original features
          Y=Y_train,                        # Labels
          Z=Z_train.astype(np.float32))    # Embeddings


Saving train embeddings to data/scarf_encodings/nslkdd_64_train.npz


# 取得test_ds的embeddings

In [54]:
'''
# 重新載入 f()（在其他地方用）
# 你必須重新建立 encoder 架構（要一樣的超參數）
encoder = Encoder(
    input_dim=train_ds.shape[1],
    dim_hidden=64,
    num_hidden=3,
    dropout=0.1
).to(device)

encoder.load_state_dict(torch.load("encoder_f.pth"))
encoder.eval()  # 推論用
'''

'\n# 重新載入 f()（在其他地方用）\n# 你必須重新建立 encoder 架構（要一樣的超參數）\nencoder = Encoder(\n    input_dim=train_ds.shape[1],\n    dim_hidden=64,\n    num_hidden=3,\n    dropout=0.1\n).to(device)\n\nencoder.load_state_dict(torch.load("encoder_f.pth"))\nencoder.eval()  # 推論用\n'

In [55]:
print(pl_scarf.scarf_model.encoder)
# ✅ 4. 取得test_ds 經過 f(x)的 embedding
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pl_scarf.to(device)
with torch.no_grad():
    x_tensor = torch.tensor(test_ds.data, dtype=torch.float32).to(device)  # PyTorch Dataset 物件轉換為 PyTorch tensor
    embeddings_test = pl_scarf.scarf_model.encoder(x_tensor).cpu().numpy() # 將從 GPU 計算得到的 embeddings tensor 複製回 CPU。這是因為後續的 numpy() 方法只能在 CPU tensor 上執行。
print("Embeddings shape:", embeddings_test.shape)

MLP(
  (0): Linear(in_features=29, out_features=64, bias=True)
  (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): Dropout(p=0.1, inplace=False)
  (4): Linear(in_features=64, out_features=64, bias=True)
  (5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): ReLU(inplace=True)
  (7): Dropout(p=0.1, inplace=False)
  (8): Linear(in_features=64, out_features=64, bias=True)
)
Embeddings shape: (25195, 64)


# 保存test .npz檔
{
  'X': ...  # 原始資料 (如 shape=[N, 64, 64, 3])
  'Y': ...  # 對應的lebal
  'Z': ...  # embeddings(如 shape=[N, 128])
}


In [56]:

# Save embeddings
output_dir = "data/scarf_encodings"
os.makedirs(output_dir, exist_ok=True)

# Get X=original Y=labels Z=embeddings
X_test= test_ds.data
Y_test= test_ds.target
Z_test= embeddings_test


# Save test split
test_path = os.path.join(output_dir, f"nslkdd_{Z_train.shape[1]}_test.npz")
print(f"Saving test embeddings to {test_path}")
np.savez(test_path,
          X=X_test,      # Original features
          Y=Y_test,                        # Labels
          Z=Z_test.astype(np.float32))    # Embeddings


Saving test embeddings to data/scarf_encodings/nslkdd_64_test.npz
